## 3. PostgreSQL -  Opérateurs, Fonctions et Procédures

In [1]:
import psycopg2
import psycopg2.extras

conn = psycopg2.connect(
    host="localhost",
    database="demo",
    user="demo_owner",
    password="OODBMS")

# query result as dict
c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

### A - Opérateurs SQL et propriétaires PostgreSQL

<div style="font-size:120%" id="create_table_sequence">
A - 1. Création d'une séquence et d'une table avec une colonne autoincrémentée
</div>

In [2]:
c.execute('DROP TABLE IF EXISTS testfuncs;')

# Création d'une séquence
c.execute('CREATE SEQUENCE testfuncs_seq AS INTEGER;')

# Création d'une table avec un champ autoincrémenté
c.execute('''CREATE TABLE testfuncs (
    i INTEGER NOT NULL DEFAULT nextval('testfuncs_seq'),
    v DECIMAL
);''')

# Suppression de la séquence en cas de suppression de la table/colonne 
c.execute('ALTER SEQUENCE testfuncs_seq OWNED BY testfuncs.i;')

conn.commit()

<div style="font-size:120%;" id="insert_from_array">
A - 2. Insertion d'une série de lignes dans la table
</div>

https://www.postgresql.org/docs/current/functions-array.html<br/>

- <code>array_fill(1,ARRAY[10])</code> crée une liste comme l'instruction python <code>[1]*10</code>
- <code>unnest()</code> transforme une liste en autant de lignes (rows)

https://mariadbtips.com/postgresql-insert-multiple-rows/#PostgreSQL_INSERT_Multiple_Rows_from_SELECT_query
- <code>INSERT INTO ... SELECT ...</code> 


In [3]:
c.execute('INSERT INTO testfuncs(v) SELECT unnest(array_fill(1,ARRAY[13]));')
c.execute('UPDATE testfuncs SET v = i-1')
conn.commit()

In [4]:
c.execute('SELECT * FROM testfuncs')
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

1, 0
2, 1
3, 2
4, 3
5, 4
6, 5
7, 6
8, 7
9, 8
10, 9
11, 10
12, 11
13, 12


<div style="font-size:120%;" id="insert_from_array">
A - 3. Quelques colonnes mathématiques...
</div>

https://www.postgresql.org/docs/current/functions-math.html

In [5]:
c.execute('ALTER TABLE testfuncs ADD COLUMN IF NOT EXISTS sqrt DECIMAL')
c.execute('UPDATE testfuncs SET sqrt = |/v')
conn.commit()

In [6]:
c.execute('ALTER TABLE testfuncs ADD COLUMN IF NOT EXISTS rad DECIMAL')
c.execute('UPDATE testfuncs SET rad = radians(30*v)')
conn.commit()

In [7]:
c.execute('ALTER TABLE testfuncs ADD COLUMN IF  NOT EXISTS cos DECIMAL')
c.execute('UPDATE testfuncs SET cos = cos(rad)')
conn.commit()

In [8]:
c.execute("SELECT v,sqrt,rad,cos FROM testfuncs")
print(' v  sqrt(v)   rad(v*30)  cos(rad)')
for d in c.fetchall(): print('{:2.0f}  {:8.6f}  {:8.6f}  {:9.6f}'.format(*d))

 v  sqrt(v)   rad(v*30)  cos(rad)
 0  0.000000  0.000000   1.000000
 1  1.000000  0.523599   0.866025
 2  1.414214  1.047198   0.500000
 3  1.732051  1.570796  -0.000000
 4  2.000000  2.094395  -0.500000
 5  2.236068  2.617994  -0.866025
 6  2.449490  3.141593  -1.000000
 7  2.645751  3.665191  -0.866025
 8  2.828427  4.188790  -0.500000
 9  3.000000  4.712389  -0.000000
10  3.162278  5.235988   0.500000
11  3.316625  5.759587   0.866025
12  3.464102  6.283185   1.000000


### B - Fonctions SQL
<div id="SQL_functions"/>

<b>N.B.</b> Pour les types <code>person_type</code> et <code>address_type</code> et pour la table <code>students</code> cf. le <a href="5_CompositeTypes.ipynb">notebook sur les types composites</a>.

In [9]:
# arguments numérotés
c.execute('''
DROP FUNCTION person_name(person_type);
CREATE FUNCTION person_name(person_type) RETURNS TEXT AS $$
    SELECT $1.first_name || ' ' || $1.last_name;
$$ LANGUAGE SQL;
''')
conn.commit()

In [10]:
c.execute('SELECT id,person,person_name(person) FROM students')
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

3, (Raymond,Deubaze), Raymond Deubaze
4, (Anna,Conda), Anna Conda
5, (Jean,Peuplu), Jean Peuplu


<div style="font-size:120%" id="named_args">
Il est également possible de nommer les arguments de la fonction&#160;:
</div>

In [11]:
# arguments nommés
c.execute('''
DROP FUNCTION person_name(person_type);
CREATE FUNCTION person_name(p person_type) RETURNS TEXT AS $$
    SELECT p.first_name || ' ' || p.last_name;
$$ LANGUAGE SQL;
''')
conn.commit()

In [12]:
c.execute('SELECT id,person,person_name(person) FROM students')
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

3, (Raymond,Deubaze), Raymond Deubaze
4, (Anna,Conda), Anna Conda
5, (Jean,Peuplu), Jean Peuplu


<div style="font-size:120%" id="on_the_fly_arg">
Appel avec un argument forgé à la volée&#160;:
</div>

In [13]:
conn.rollback()
c.execute("SELECT person_name(ROW('Alex','Terrieur'))")
print(c.fetchone()[0])

Alex Terrieur


<div style="font-size:120%" id="select_func">
Fonction effectuant un select&#160;:
</div>

In [14]:
c.execute('ALTER TABLE testfuncs ADD COLUMN IF NOT EXISTS fibo INTEGER')

c.execute('''
CREATE OR REPLACE FUNCTION compute_fibo(n decimal) RETURNS DECIMAL AS $$
    SELECT SUM(fibo) FROM testfuncs WHERE i=n-1 OR i=n-2
$$ LANGUAGE SQL;
''')

c.execute('UPDATE testfuncs SET fibo=0')
c.execute('UPDATE testfuncs SET fibo=1 WHERE i=2')
c.execute('UPDATE testfuncs SET fibo = compute_fibo(i) WHERE i > 2')
conn.commit()

In [15]:
c.execute("SELECT * FROM testfuncs ORDER BY i")
print(' i   v  sqrt(v)   rad(v*30)  cos(rad)  fibo(i)')
for d in c.fetchall(): print('{:2d}  {:2.0f}  {:8.6f}  {:8.6f}  {:9.6f} {:4d}'.format(*d))

 i   v  sqrt(v)   rad(v*30)  cos(rad)  fibo(i)
 1   0  0.000000  0.000000   1.000000    0
 2   1  1.000000  0.523599   0.866025    1
 3   2  1.414214  1.047198   0.500000    1
 4   3  1.732051  1.570796  -0.000000    2
 5   4  2.000000  2.094395  -0.500000    3
 6   5  2.236068  2.617994  -0.866025    5
 7   6  2.449490  3.141593  -1.000000    8
 8   7  2.645751  3.665191  -0.866025   13
 9   8  2.828427  4.188790  -0.500000   21
10   9  3.000000  4.712389  -0.000000   34
11  10  3.162278  5.235988   0.500000   55
12  11  3.316625  5.759587   0.866025   89
13  12  3.464102  6.283185   1.000000  144


<div style="font-size:120%" id="procedure">
Définition d'une procédure modifiant la table <code>testfuncs</code> :
</div>

In [16]:
c.execute('''
CREATE OR REPLACE PROCEDURE testfuncs_extend(INTEGER) AS $$
    INSERT INTO testfuncs(fibo) SELECT unnest(array_fill(0,ARRAY[$1]));
    UPDATE testfuncs
        SET
            v = i-1,
            sqrt = |/i,
            rad = radians(30*(i-1)),
            cos = cos(radians(30*(i-1))),
            fibo = compute_fibo(i)
        WHERE i > 2;
$$ LANGUAGE SQL;
''')
conn.commit()

In [17]:
c.execute('DELETE FROM testfuncs WHERE i > 13')
c.execute('ALTER SEQUENCE testfuncs_seq RESTART WITH 14')

c.execute('SELECT COUNT(*) FROM testfuncs')
print(c.fetchone()[0])

c.execute('CALL testfuncs_extend(12)')

c.execute('SELECT COUNT(*) FROM testfuncs')
print(c.fetchone()[0])

13
25


In [18]:
c.execute("SELECT * FROM testfuncs WHERE i > 12 ORDER BY i")
print(' i   v  sqrt(v)    rad(v*30)  cos(rad)  fibo(i)')
for d in c.fetchall(): print('{:2d}  {:2.0f}  {:8.6f}  {:9.6f}  {:9.6f}   {:5d}'.format(*d))

 i   v  sqrt(v)    rad(v*30)  cos(rad)  fibo(i)
13  12  3.605551   6.283185   1.000000     144
14  13  3.741657   6.806784   0.866025     233
15  14  3.872983   7.330383   0.500000     377
16  15  4.000000   7.853982   0.000000     610
17  16  4.123106   8.377580  -0.500000     987
18  17  4.242641   8.901179  -0.866025    1597
19  18  4.358899   9.424778  -1.000000    2584
20  19  4.472136   9.948377  -0.866025    4181
21  20  4.582576  10.471976  -0.500000    6765
22  21  4.690416  10.995574  -0.000000   10946
23  22  4.795832  11.519173   0.500000   17711
24  23  4.898979  12.042772   0.866025   28657
25  24  5.000000  12.566371   1.000000   46368


<div style="font-size:120%" id="func_returning_table">
Une fonction peut retourner une donnée composée ou un enregistrement, issu d'une table ou non. Mieux, il est également possible de renvoyer <b>plusieurs</b> lignes de tables&#160;:
</div>

In [19]:
c.execute('''
DROP FUNCTION get_trigonometric_period();
CREATE FUNCTION get_trigonometric_period()
RETURNS TABLE( deg DECIMAL, rad DECIMAL, cos DECIMAL, sin DECIMAL ) AS $$
    SELECT degrees(rad) as deg, rad, cos, sin(rad) FROM testfuncs WHERE degrees(rad) <= 361;
$$ LANGUAGE SQL;
''')
conn.commit()

In [20]:
c.execute('SELECT * FROM get_trigonometric_period()')
print('{}    {}        {}       {}'.format(*[d[0] for d in c.description]))
for d in c.fetchall(): print('{:3.0f}  {:8.6f}  {:9.6f} {:9.6f}'.format(*d))

deg    rad        cos       sin
  0  0.000000   1.000000  0.000000
 30  0.523599   0.866025  0.500000
 60  1.047198   0.500000  0.866025
 90  1.570796   0.000000  1.000000
120  2.094395  -0.500000  0.866025
150  2.617994  -0.866025  0.500000
180  3.141593  -1.000000  0.000000
210  3.665191  -0.866025 -0.500000
240  4.188790  -0.500000 -0.866025
270  4.712389  -0.000000 -1.000000
300  5.235988   0.500000 -0.866025
330  5.759587   0.866025 -0.500000
360  6.283185   1.000000  0.000000
